# Export variance contributed by different sources

In [79]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [80]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [81]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites, images and settings

In [82]:
#definet the color pallette
vibgYOR = ['330044', '220066', '1133cc', '33dd00', 'ffda21', 'ff6622', 'd10000']
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec");
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)
# generete the pixel area map
pixelArea = ee.Image.pixelArea().divide(10000) # to ha unit
# load the biome layer
biomeLayer = compositeImage.select("WWF_Biome")
biomeMask = biomeLayer.mask(biomeLayer.neq(98)).mask(biomeLayer.neq(99)).gt(0)
# load the mean maps for present and potential
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the biomass density layers
mergedAGB_PresentMean = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Present_density_Ensambled_Mean_20230427").unmask() 
mergedAGB_PotentialMean =  ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()
# define the standardized projection
stdProj = mergedAGB_PresentMean.projection()
# load the two forest cover layer for existing and potential forest
presentForestCover = compositeImage.select('PresentTreeCover').unmask()# make sure it's in  0-1 scale
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask() # make sure it's in  0-1 scale

# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialForestCover.gt(0)

In [50]:
# # load the carbon density layers
# SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
# SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# # mask the diffrence layer
# SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0))

# # load the present and potential forest cover
# presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
# potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# # define the present and potential forest cover masks
# presentMask = presentForestCover.gt(0)
# potentialMask = potentialCoverAdjusted.gte(0.1)

# # calculate the sum of the potential in soil with the consideration of forest cover
# SandermannCarbonStockLoss = SandermannCarbonLoss.multiply(pixelArea).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

# # add the soil into the PGB as the total potential
# potentialTotal_Abs = potentialPGB_Abs.add(SandermannCarbonStockLoss)
# # compose those bands into an image
# lowerUpperImage = presentAGB_Lower_Abs.rename('preAGB_Lower').addBands(presentAGB_Upper_Abs.rename('preAGB_Upper')).addBands(potentialAGB_Lower_Abs.rename('potAGB_Lower')).addBands(potentialAGB_Upper_Abs.rename('potAGB_Upper')).addBands(presentRoot_Lower_Abs.rename('preRoot_Lower')).addBands(presentRoot_Upper_Abs.rename('preRoot_Upper')).addBands(potentialRoot_Lower_Abs.rename('potRoot_Lower')).addBands(potentialRoot_Upper_Abs.rename('potRoot_Upper')).addBands(presentLitter_Lower_Abs.rename('preLitter_Lower')).addBands(presentLitter_Upper_Abs.rename('preLitter_Upper')).addBands(potentialLitter_Lower_Abs.rename('potLitter_Lower')).addBands(potentialLitter_Upper_Abs.rename('potLitter_Upper')).addBands(potentialTotal_Abs.rename('PotentialTotal'))

## 2 Load the images of the uncertainties

In [51]:
# load the images with uncertainty
GS1_MaxScaler_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/GS1_MaxScaler_Lower_Upper_Map')
GS1_MeanScaler_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/GS1_MeanScaler_Lower_Upper_Map')

GS2_MaxScaler_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/GS2_MaxScaler_Lower_Upper_Map')
GS2_MeanScaler_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/GS2_MeanScaler_Lower_Upper_Map')

HM1_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/HM1_Lower_Upper_Map')
HM2_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/HM2_Lower_Upper_Map')

SD1_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/SD1_Lower_Upper_Map')
SD2_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/SD2_Lower_Upper_Map')

WK1_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/WK1_Lower_Upper_Map')
WK2_Image = ee.Image('users/leonidmoore/ForestBiomass/UncertaintyFigure/WK2_Lower_Upper_Map')


## 3 Calculate the uncertianty range maps

In [52]:
# this is the AGB range
GS1_MaxScaler_AGB_Range = GS1_MaxScaler_Image.select('potAGB_Upper').subtract(GS1_MaxScaler_Image.select('potAGB_Lower'))
GS1_MeanScaler_AGB_Range = GS1_MeanScaler_Image.select('potAGB_Upper').subtract(GS1_MeanScaler_Image.select('potAGB_Lower'))

GS2_MaxScaler_AGB_Range = GS2_MaxScaler_Image.select('potAGB_Upper').subtract(GS2_MaxScaler_Image.select('potAGB_Lower'))
GS2_MeanScaler_AGB_Range = GS2_MeanScaler_Image.select('potAGB_Upper').subtract(GS2_MeanScaler_Image.select('potAGB_Lower'))

HM1_AGB_Range = HM1_Image.select('potAGB_Upper').subtract(HM1_Image.select('potAGB_Lower'))
HM2_AGB_Range = HM2_Image.select('potAGB_Upper').subtract(HM2_Image.select('potAGB_Lower'))

SD1_AGB_Range = SD1_Image.select('potAGB_Upper').subtract(SD1_Image.select('potAGB_Lower'))
SD2_AGB_Range = SD2_Image.select('potAGB_Upper').subtract(SD2_Image.select('potAGB_Lower'))

WK1_AGB_Range = WK1_Image.select('potAGB_Upper').subtract(WK1_Image.select('potAGB_Lower'))
WK2_AGB_Range = WK2_Image.select('potAGB_Upper').subtract(WK2_Image.select('potAGB_Lower'))

GS_AGB_Range = GS1_MaxScaler_AGB_Range.add(GS2_MaxScaler_AGB_Range).add(GS1_MeanScaler_AGB_Range).add(GS2_MeanScaler_AGB_Range).divide(4)
RM_AGB_Range = HM1_AGB_Range.add(HM2_AGB_Range).add(SD1_AGB_Range).add(SD2_AGB_Range).add(WK1_AGB_Range).add(WK2_AGB_Range).divide(6)

In [53]:
# This is the root carbon stock range
GS1_MaxScaler_Root_Range = GS1_MaxScaler_Image.select('potRoot_Upper').subtract(GS1_MaxScaler_Image.select('potRoot_Lower'))
GS1_MeanScaler_Root_Range = GS1_MeanScaler_Image.select('potRoot_Upper').subtract(GS1_MeanScaler_Image.select('potRoot_Lower'))

GS2_MaxScaler_Root_Range = GS2_MaxScaler_Image.select('potRoot_Upper').subtract(GS2_MaxScaler_Image.select('potRoot_Lower'))
GS2_MeanScaler_Root_Range = GS2_MeanScaler_Image.select('potRoot_Upper').subtract(GS2_MeanScaler_Image.select('potRoot_Lower'))

HM1_Root_Range = HM1_Image.select('potRoot_Upper').subtract(HM1_Image.select('potRoot_Lower'))
HM2_Root_Range = HM2_Image.select('potRoot_Upper').subtract(HM2_Image.select('potRoot_Lower'))

SD1_Root_Range = SD1_Image.select('potRoot_Upper').subtract(SD1_Image.select('potRoot_Lower'))
SD2_Root_Range = SD2_Image.select('potRoot_Upper').subtract(SD2_Image.select('potRoot_Lower'))

WK1_Root_Range = WK1_Image.select('potRoot_Upper').subtract(WK1_Image.select('potRoot_Lower'))
WK2_Root_Range = WK2_Image.select('potRoot_Upper').subtract(WK2_Image.select('potRoot_Lower'))

GS_Root_Range = GS1_MaxScaler_Root_Range.add(GS2_MaxScaler_Root_Range).add(GS1_MeanScaler_Root_Range).add(GS2_MeanScaler_Root_Range).divide(4)
RM_Root_Range = HM1_Root_Range.add(HM2_Root_Range).add(SD1_Root_Range).add(SD2_Root_Range).add(WK1_Root_Range).add(WK2_Root_Range).divide(6)

In [54]:
# This is the Litter carbon stock range
GS1_MaxScaler_Litter_Range = GS1_MaxScaler_Image.select('potLitter_Upper').subtract(GS1_MaxScaler_Image.select('potLitter_Lower'))
GS1_MeanScaler_Litter_Range = GS1_MeanScaler_Image.select('potLitter_Upper').subtract(GS1_MeanScaler_Image.select('potLitter_Lower'))

GS2_MaxScaler_Litter_Range = GS2_MaxScaler_Image.select('potLitter_Upper').subtract(GS2_MaxScaler_Image.select('potLitter_Lower'))
GS2_MeanScaler_Litter_Range = GS2_MeanScaler_Image.select('potLitter_Upper').subtract(GS2_MeanScaler_Image.select('potLitter_Lower'))

HM1_Litter_Range = HM1_Image.select('potLitter_Upper').subtract(HM1_Image.select('potLitter_Lower'))
HM2_Litter_Range = HM2_Image.select('potLitter_Upper').subtract(HM2_Image.select('potLitter_Lower'))

SD1_Litter_Range = SD1_Image.select('potLitter_Upper').subtract(SD1_Image.select('potLitter_Lower'))
SD2_Litter_Range = SD2_Image.select('potLitter_Upper').subtract(SD2_Image.select('potLitter_Lower'))

WK1_Litter_Range = WK1_Image.select('potLitter_Upper').subtract(WK1_Image.select('potLitter_Lower'))
WK2_Litter_Range = WK2_Image.select('potLitter_Upper').subtract(WK2_Image.select('potLitter_Lower'))

GS_Litter_Range = GS1_MaxScaler_Litter_Range.add(GS2_MaxScaler_Litter_Range).add(GS1_MeanScaler_Litter_Range).add(GS2_MeanScaler_Litter_Range).divide(4)
RM_Litter_Range = HM1_Litter_Range.add(HM2_Litter_Range).add(SD1_Litter_Range).add(SD2_Litter_Range).add(WK1_Litter_Range).add(WK2_Litter_Range).divide(6)

In [55]:
SandermannCarbonLossLower = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU_Lower").unmask()
SandermannCarbonLossUpper = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU_Upper").unmask()
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialCoverAdjusted.gte(0.1)

soilLower = SandermannCarbonLossLower.multiply(pixelArea).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)
SoilUpper = SandermannCarbonLossUpper.multiply(pixelArea).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

Soil_Range = SoilUpper.subtract(soilLower)

In [56]:
# calculate the pixel level model type uncertainty range
GS_Model1_Total = GS1_MaxScaler_Image.select('PotentialTotal').add(GS1_MeanScaler_Image.select('PotentialTotal')).divide(2)
GS_Model2_Total = GS2_MaxScaler_Image.select('PotentialTotal').add(GS2_MeanScaler_Image.select('PotentialTotal')).divide(2)
GS_Model_Range = GS_Model1_Total.subtract(GS_Model2_Total).abs() # abs to get the range 

RM_Model1_Total = HM1_Image.select('PotentialTotal').add(SD1_Image.select('PotentialTotal')).add(WK1_Image.select('PotentialTotal')).divide(3)
RM_Model2_Total = HM2_Image.select('PotentialTotal').add(SD2_Image.select('PotentialTotal')).add(WK2_Image.select('PotentialTotal')).divide(3)
RM_Model_Range = RM_Model1_Total.subtract(RM_Model2_Total).abs()

In [57]:
# calculate the pixel level model type uncertainty range
GS_Data1_Total_Max = GS1_MaxScaler_Image.select('PotentialTotal').addBands(GS1_MeanScaler_Image.select('PotentialTotal')).reduce(ee.Reducer.max())
GS_Data1_Total_Min = GS1_MaxScaler_Image.select('PotentialTotal').addBands(GS1_MeanScaler_Image.select('PotentialTotal')).reduce(ee.Reducer.min())
GS_Data1_Total_Range = GS_Data1_Total_Max.subtract(GS_Data1_Total_Min)

GS_Data2_Total_Max = GS2_MaxScaler_Image.select('PotentialTotal').addBands(GS2_MeanScaler_Image.select('PotentialTotal')).reduce(ee.Reducer.max())
GS_Data2_Total_Min = GS2_MaxScaler_Image.select('PotentialTotal').addBands(GS2_MeanScaler_Image.select('PotentialTotal')).reduce(ee.Reducer.min())
GS_Data2_Total_Range = GS_Data2_Total_Max.subtract(GS_Data2_Total_Min)

GS_Data_Total_Range = GS_Data1_Total_Range.add(GS_Data2_Total_Range).divide(2)


RM_Data1_Total_Max = HM1_Image.select('PotentialTotal').addBands(SD1_Image.select('PotentialTotal')).addBands(WK1_Image.select('PotentialTotal')).reduce(ee.Reducer.max())
RM_Data1_Total_Min = HM1_Image.select('PotentialTotal').addBands(SD1_Image.select('PotentialTotal')).addBands(WK1_Image.select('PotentialTotal')).reduce(ee.Reducer.min())
RM_Data1_Total_Range = RM_Data1_Total_Max.subtract(RM_Data1_Total_Min)

RM_Data2_Total_Max = HM2_Image.select('PotentialTotal').addBands(SD2_Image.select('PotentialTotal')).addBands(WK2_Image.select('PotentialTotal')).reduce(ee.Reducer.max())
RM_Data2_Total_Min = HM2_Image.select('PotentialTotal').addBands(SD2_Image.select('PotentialTotal')).addBands(WK2_Image.select('PotentialTotal')).reduce(ee.Reducer.min())
RM_Data2_Total_Range = RM_Data2_Total_Max.subtract(RM_Data2_Total_Min)

RM_Data_Total_Range = RM_Data1_Total_Range.add(RM_Data2_Total_Range).divide(2)


In [58]:
fullRangeGS = GS_Data_Total_Range.add(GS_Model_Range).add(Soil_Range).add(GS_Litter_Range).add(GS_Root_Range).add(GS_AGB_Range)
# GS uncertainty contribution
contributionImageGS = GS_Data_Total_Range.addBands(GS_Model_Range).addBands(Soil_Range).addBands(GS_Litter_Range).addBands(GS_Root_Range).addBands(GS_AGB_Range).divide(fullRangeGS).rename(['Data','Model','Soil','Litter','Root','AGB'])

In [59]:
fullRangeRM = RM_Data_Total_Range.add(RM_Model_Range).add(Soil_Range).add(RM_Litter_Range).add(RM_Root_Range).add(RM_AGB_Range)
# RM uncertainty contribution
contributionImageRM =  RM_Data_Total_Range.addBands(RM_Model_Range).addBands(Soil_Range).addBands(RM_Litter_Range).addBands(RM_Root_Range).addBands(RM_AGB_Range).divide(fullRangeRM).rename(['Data','Model','Soil','Litter','Root','AGB'])

In [60]:
exportGS_Uncertainty = ee.batch.Export.image.toAsset(image = contributionImageGS,
                                               description = 'GS_Uncertainty_Contribution_Maps_Export',
                                               assetId = 'users/leonidmoore/ForestBiomass/UncertaintyFigure/GS_Uncertainty_contribution_Maps',
                                               region = unboundedGeo,
                                               crs = 'EPSG:4326',
                                               crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                               maxPixels = 1e13)
# start the export task
exportGS_Uncertainty.start()
# show the task status
exportGS_Uncertainty.status()

{'state': 'READY',
 'description': 'GS_Uncertainty_Contribution_Maps_Export',
 'creation_timestamp_ms': 1690818102463,
 'update_timestamp_ms': 1690818102463,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'TP57UZLSREOUMVUTSVDPSVAP',
 'name': 'projects/earthengine-legacy/operations/TP57UZLSREOUMVUTSVDPSVAP'}

In [61]:
exportRM_Uncertainty = ee.batch.Export.image.toAsset(image = contributionImageRM,
                                               description = 'RM_Uncertainty_Contribution_Maps_Export',
                                               assetId = 'users/leonidmoore/ForestBiomass/UncertaintyFigure/RM_Uncertainty_contribution_Maps',
                                               region = unboundedGeo,
                                               crs = 'EPSG:4326',
                                               crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                               maxPixels = 1e13)
# start the export task
exportRM_Uncertainty.start()
# show the task status
exportRM_Uncertainty.status()

{'state': 'READY',
 'description': 'RM_Uncertainty_Contribution_Maps_Export',
 'creation_timestamp_ms': 1690818103944,
 'update_timestamp_ms': 1690818103944,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'JYCK7XHJUZUCFV2LN3RSIEIY',
 'name': 'projects/earthengine-legacy/operations/JYCK7XHJUZUCFV2LN3RSIEIY'}

## 4 Export the top two maximum 

### 4.1 Export for the Satellite-derived model

In [83]:
# refer to the code in GEE accout '20230802_variance_explained_by_different_parts'


# load the image with the contribution of the variance from all sources
rawImage = ee.Image("users/leonidmoore/ForestBiomass/UncertaintyFigure/RM_Uncertainty_contribution_Maps");
# re-sort the image by the band names with alphabetical order
varianceImage = rawImage.select(["Model","Data","AGB","Root","Litter","Soil"]);
# print the new band names
print(varianceImage.bandNames().getInfo())
# transfer the image into array
arrayImage = varianceImage.toArray()
# sort them by the values in a ascending way(default in GEE)
sortedArrayImage = arrayImage.arraySort()
# slice the array to the get top two values
maxValImageRM = sortedArrayImage.arraySlice(0,-2).arrayFlatten([["SecondMax","FirstMax"]])

# now let's export the correpoding IDs for the max and second mas value
# generate a multi-band image with the order index which will be used for pointing at the relavant sources
def bandfunc(bandOrder):
    return ee.Image.constant (bandOrder)

idImage = ee.ImageCollection(ee.List.sequence(0,rawImage.bandNames().length().subtract(1)).map(bandfunc)).toBands()
print(idImage.bandNames().getInfo())
# sort the idImage by the value of the array image
sortedArrayImage = idImage.toArray().arraySort(arrayImage)
maxValID_ImageRM = sortedArrayImage.arraySlice(0,-2).arrayFlatten([["SecondMax","FirstMax"]]);
print(maxValID_Image.bandNames().getInfo())

['Model', 'Data', 'AGB', 'Root', 'Litter', 'Soil']
['0_constant', '1_constant', '2_constant', '3_constant', '4_constant', '5_constant']
['SecondMax', 'FirstMax']


In [84]:
# export the maps into google cloud storage
exportVarianceValue = ee.batch.Export.image.toCloudStorage(image = maxValImageRM,
                                                           description = 'RM_maxValImage_Map_Export',
                                                           fileNamePrefix = 'ForestBiomassVariance/RM_max_variance_values_map',
                                                           region = unboundedGeo,
                                                           bucket = "crowtherlab_gcsb_lidong",
                                                           crs = 'EPSG:4326',
                                                           crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                           maxPixels = 1e13,
                                                           fileFormat = 'GeoTIFF')
# start the export task
exportVarianceValue.start()
# show the task status
exportVarianceValue.status()

# export the maps into google cloud storage
exportVarianceID = ee.batch.Export.image.toCloudStorage(image = maxValID_ImageRM,
                                                        description = 'RM_maxID_Image_Map_Export',
                                                        fileNamePrefix = 'ForestBiomassVariance/RM_max_variance_IDs_map',
                                                        region = unboundedGeo,
                                                        bucket = "crowtherlab_gcsb_lidong",
                                                        crs = 'EPSG:4326',
                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                        maxPixels = 1e13,
                                                        fileFormat = 'GeoTIFF')
# start the export task
exportVarianceID.start()
# show the task status
exportVarianceID.status()

{'state': 'READY',
 'description': 'RM_maxID_Image_Map_Export',
 'creation_timestamp_ms': 1691007745739,
 'update_timestamp_ms': 1691007745739,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'ENJZZWGY4GLJVHNAIPD4AOAF',
 'name': 'projects/earthengine-legacy/operations/ENJZZWGY4GLJVHNAIPD4AOAF'}

### 4.2 Export for the Ground-sourced model

In [85]:
# refer to the code in GEE accout '20230802_variance_explained_by_different_parts'


# load the image with the contribution of the variance from all sources
rawImage = ee.Image("users/leonidmoore/ForestBiomass/UncertaintyFigure/GS_Uncertainty_contribution_Maps");
# re-sort the image by the band names with alphabetical order
varianceImage = rawImage.select(["Model","Data","AGB","Root","Litter","Soil"]);
# print the new band names
print(varianceImage.bandNames().getInfo())
# transfer the image into array
arrayImage = varianceImage.toArray()
# sort them by the values in a ascending way(default in GEE)
sortedArrayImage = arrayImage.arraySort()
# slice the array to the get top two values
maxValImageGS = sortedArrayImage.arraySlice(0,-2).arrayFlatten([["SecondMax","FirstMax"]])

# now let's export the correpoding IDs for the max and second mas value
# generate a multi-band image with the order index which will be used for pointing at the relavant sources
def bandfunc(bandOrder):
    return ee.Image.constant (bandOrder)

idImage = ee.ImageCollection(ee.List.sequence(0,rawImage.bandNames().length().subtract(1)).map(bandfunc)).toBands()
print(idImage.bandNames().getInfo())
# sort the idImage by the value of the array image
sortedArrayImage = idImage.toArray().arraySort(arrayImage)
maxValID_ImageGS = sortedArrayImage.arraySlice(0,-2).arrayFlatten([["SecondMax","FirstMax"]]);
print(maxValID_Image.bandNames().getInfo())

['Model', 'Data', 'AGB', 'Root', 'Litter', 'Soil']
['0_constant', '1_constant', '2_constant', '3_constant', '4_constant', '5_constant']
['SecondMax', 'FirstMax']


In [86]:
# export the maps into google cloud storage
exportVarianceValue = ee.batch.Export.image.toCloudStorage(image = maxValImageGS,
                                                           description = 'GS_maxValImage_Map_Export',
                                                           fileNamePrefix = 'ForestBiomassVariance/GS_max_variance_values_map',
                                                           region = unboundedGeo,
                                                           bucket = "crowtherlab_gcsb_lidong",
                                                           crs = 'EPSG:4326',
                                                           crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                           maxPixels = 1e13,
                                                           fileFormat = 'GeoTIFF')
# start the export task
exportVarianceValue.start()
# show the task status
exportVarianceValue.status()

# export the maps into google cloud storage
exportVarianceID = ee.batch.Export.image.toCloudStorage(image = maxValID_ImageGS,
                                                        description = 'GS_maxID_Image_Map_Export',
                                                        fileNamePrefix = 'ForestBiomassVariance/GS_max_variance_IDs_map',
                                                        region = unboundedGeo,
                                                        bucket = "crowtherlab_gcsb_lidong",
                                                        crs = 'EPSG:4326',
                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                        maxPixels = 1e13,
                                                        fileFormat = 'GeoTIFF')
# start the export task
exportVarianceID.start()
# show the task status
exportVarianceID.status()

{'state': 'READY',
 'description': 'GS_maxID_Image_Map_Export',
 'creation_timestamp_ms': 1691007749371,
 'update_timestamp_ms': 1691007749371,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '2LEAKXZV3N2CS3X756AXYLEB',
 'name': 'projects/earthengine-legacy/operations/2LEAKXZV3N2CS3X756AXYLEB'}